In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http://securit

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...|2015-08-31 00:00:00|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...| 

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3W4P9UBGNGH1U|          2|            1|          3|   N|                Y|
|R15V54KBMTQWAY|          4|            0|          0|   N|                N|
| RY8I449HNXSVF|          5|            0|          0|   N|                Y|
|R18TLJYCKJFLSR|          5|            0|          0|   N|                Y|
|R1NK26SWS53B8Q|          5|            0|          0|   N|                Y|
|R11LOHEDYJALTN|          5|            0|          0|   N|                Y|
|R3ALQVQB2P9LA7|          4|            0|          0|   N|                Y|
|R3MWLXLNO21PDQ|          5|            0|          0|   N|                Y|
|R2L15IS24CX0LI|          5|            0|          0|   N|                Y|
|R1DJ8976WPWVZU|          3|            0|          0|   N|     

In [12]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count is equal to or greater than 20
total_votes=vine_df.filter('total_votes>=20')
total_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|
|R111DJA10Y6CMU|          5|           26|         39|   N|                Y|
|R2EE2TR4MRDV0U|          5|           53|         59|   N|                Y|
| RD4A80I5JDHED|          5|           69|         77|   N|                N|
|R1GU6IYZQWQE8X|          2|           24|         25|   N|     

In [23]:
# Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
votes_percentages = total_votes.withColumn('votes_percentages',total_votes['helpful_votes']/total_votes['total_votes']).filter('votes_percentages>.50')
votes_percentages.show()


+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase| votes_percentages|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|0.9152542372881356|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|0.7142857142857143|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|0.9523809523809523|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|0.6363636363636364|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|0.9574468085106383|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|0.9423076923076923|
|R111DJA10Y6CMU|          5|           26|         39|   N|                Y|0.6666666666666666|
|R2EE2TR4MRDV0U|          5|  

In [26]:
# Filter above Dataframe and retrieve all the rows where the review was not part of the Vine program (unpaid)
vine_unpaid = votes_percentages.filter(votes_percentages['vine']=='N')
vine_unpaid.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase| votes_percentages|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R2WOW0TURNXB26|          3|           54|         59|   N|                Y|0.9152542372881356|
|R13VL62Y2HBQ0B|          5|           15|         21|   N|                Y|0.7142857142857143|
|R22G55KAPZKJQV|          4|           20|         21|   N|                Y|0.9523809523809523|
|R1610PGTJS7G3N|          2|           28|         44|   N|                Y|0.6363636363636364|
| RLQL04BL0QXOJ|          4|           45|         47|   N|                Y|0.9574468085106383|
|R2AYJHH8WJNGAU|          4|           98|        104|   N|                Y|0.9423076923076923|
|R111DJA10Y6CMU|          5|           26|         39|   N|                Y|0.6666666666666666|
|R2EE2TR4MRDV0U|          5|  

In [33]:
from pyspark.sql.functions import col,when,count,lit
total_reviews_df = votes_percentages.groupBy("vine").agg(
    count(col("vine")).alias("Total Reviews"),
    count(when(col("star_rating") == 5, True)).alias("Total 5 Star Reviews"),
    (count(when(col("star_rating") == 5, True))/count(col("vine"))*100).alias("Percentage 5 Star Total Review")).show()

+----+-------------+--------------------+------------------------------+
|vine|Total Reviews|Total 5 Star Reviews|Percentage 5 Star Total Review|
+----+-------------+--------------------+------------------------------+
|   Y|          613|                 222|            36.215334420880914|
|   N|        64771|               30523|             47.12448472310139|
+----+-------------+--------------------+------------------------------+

